In [2]:
import re
import pandas as pd
import MySQLdb
import sys, csv, os
sys.path.append("D:/DataBaseUpdate/To_clone")
from ConfigFiles import config #this attaches my database credentials

In [5]:
academic = []
government = []
corporate = []
hospital = []

lookup = {'Academic': academic, 'Government': government, "Corporate": corporate, 'Hospital': hospital}

def add_to_list(item, category):
    lookup[category].append(item)

with open('AcadCorpGovIndiv.txt', 'rb') as myfile:
    category = None
    for line in myfile.readlines():
        if line.startswith("*"):
            category = line[2:].strip("\r\n")
            if category == 'People':
                break
        if not line.startswith("*"):
            add_to_list(line[6:].strip("\r\n"), category)

In [6]:
government += ["represented", "Department of","United States of America", "The National Institute of Standards and Technology",
               "National Institutes of Health"]
academic += ['Massachusetts Institute of Technology', 'California Institute of Technology','Rensselaer Polytechnic Institute']
hospital += ['Sloan-Kettering Institute for Cancer Research']
corporate += ['Battelle Memorial Institute', 'The Scripps Research Institute','The Salk Institute for Biological Studies','Dana-Farber Cancer Institute, Inc.', 'Midwest Research Institute', 'Southwest Research Institute', 'Whitehead Institute for Biomedical Research']

In [30]:
academic_regex = re.compile("|".join(academic), re.IGNORECASE)

In [31]:
government_regex = re.compile("|".join(government), re.IGNORECASE)

In [32]:
corporate_regex = re.compile("|".join(corporate), re.IGNORECASE)
hospital_regex = re.compile("|".join(hospital), re.IGNORECASE)

In [39]:
def org_types(text):
    if text == "":
        return "Person"
    academic = bool(re.search(academic_regex, text))
    government = bool(re.search(government_regex, text))
    corporate = bool(re.search(corporate_regex, text))
    hospital = bool(re.search(hospital_regex, text))
    total = academic + government + corporate + hospital
    if total > 1:
        return "Ambiguous"
    if academic: 
        return "Academic"
    if government:
        return "Government"
    if corporate:
        return "Corporate"
    if hospital:
        return "Hospital"
    else:
        return "Corporate"

In [40]:
data = pd.read_csv('F:\Govt_Int\Final_CSVS/assignee_type.csv')
data['organization'] = data['organization'].fillna("")
data = data.dropna()
data['thes_types'] = data['organization'].apply(lambda x : org_types(x))

In [41]:
#data["Inst"] = data['organization'].apply(lambda x : True if 'Institute' in x else False)
#data[data["Inst"]==True].groupby('organization').max().sort_values("thes_types", ascending=False)

In [42]:
data.to_csv('F:\Govt_Int\Final_CSVS/assignees_lookedup_types.csv')

### Check the most common assignees

In [3]:
data = pd.read_csv('F:\Govt_Int\Final_CSVS/assignees_lookedup_types.csv')
data.head()

C:\Program Files\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,patent_id,assignee_type,organization,thes_types
0,0,3930295,2.0,Signetics Corporation,Corporate
1,1,3930295,2.0,Watkins-Johnson,Corporate
2,2,3930317,6.0,The United States of America as represented by...,Government
3,3,3930366,2.0,General Motors Corporation,Corporate
4,4,3930405,0.0,The United States of America as represented by...,Government


In [22]:
counts = data.groupby("organization").agg({'patent_id':'count', 'thes_types':'min'})
to_check = counts[counts['patent_id'] >20]
#to_check['thes_types'].value_counts()

In [23]:
to_check.reset_index(inplace=True)
to_check.columns = ['organization', 'type', 'count']
to_check.to_csv('F:/Govt_Int/ManualCheck/top_orgs_to_check.csv', index = False)

### Use the regex to classify all the assignees

In [43]:
all_assignees = pd.read_csv("F:/Govt_Int/Final_CSVS/all_assignees.csv").fillna("")
all_assignees['thes_types'] = all_assignees['organization'].apply(lambda x : org_types(x))
all_assignees.head()

,id,type,name_first,name_last,organization,thes_types
0,00003f5c5d92354570b9bae01b3e7272,0,,,"E-Z Anchor Bolt Template, Inc.",Corporate
1,000058159cdbf11415ffad8001c192f6,0,,,Roll and Go AG,Corporate
2,00005a7b722c0d70af7c0f6706cdc067,0,,,PERSCITUS INNOVATIONS LLC,Corporate
3,00007b72e8f46f63797266c63ad3a34f,0,,,"Fleetwood Aluminum Products, Inc.",Corporate
4,000099cec50350524fc0704f0cdd8db6,0,,,B.V. Nederlandse Industrie Van Eiprodukten,Corporate


In [45]:
all_assignees['thes_types'].value_counts()

Corporate     328532
Person         40483
Academic        4142
Government      1734
Ambiguous       1617
Hospital         405
Name: thes_types, dtype: int64

In [46]:
mydb = MySQLdb.connect(config.host, config.username, config.password, 'patent_20170808')

In [49]:
#only run this once to upload the data
all_assignees.to_sql(con=mydb, name='assignees_by_type', if_exists='replace', flavor='mysql')

C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 67
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 171
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 179
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 273
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data truncated for column 'organization' at row 326
  conn.executemany(self.insert_statement(), data_list)
C:\Program Files\Anaconda2\lib\site-packages\pandas\io\sql.py:1445: Warning: Data